In [1]:
import os
from pathlib import Path
import numpy as np
import yaml
from tqdm import tqdm
import torch
import sys
sys.path.append('../')

In [2]:
from models.models import *
from utils.datasets import create_dataloader
from utils.general import non_max_suppression, increment_path, check_dataset, check_img_size
from utils.torch_utils import select_device
from utils.metrics import ap_per_class, compute_ap
from feature_mining.utils import *

In [3]:
# Set parameters
%matplotlib inline
imgsz=1600
imgsz_test=1600
# img_size = (1600, 900)
batch_size=48
gs=64
cache_image=True
rect=True
pad=0.5
conf_thres=0.001
iou_thres=0.6
cfg = '../cfg/yolov4-tiny-25.cfg'
device_str = 'cpu'
weights='../weights/best.pt'
data='../data/nuimages_rider.yaml'
save_dir = Path(increment_path(Path('runs/visualize') / 'yolov4-tiny-25', exist_ok=True))  # increment run
save_dir.mkdir(parents=True, exist_ok=True)

In [4]:
print(torch.cuda.is_available())
import sys
print("Python version")
print (sys.version)
print("Version info.")
print (sys.version_info)
assert torch.cuda.is_available(), 'CUDA unavailable, invalid device %s requested' % device

True
Python version
3.8.10 (default, Jun 22 2022, 20:18:18) 
[GCC 9.4.0]
Version info.
sys.version_info(major=3, minor=8, micro=10, releaselevel='final', serial=0)


### Load Model

In [10]:
from models import models

# device = select_device(device_str)
device = 'cuda:1'

# Load with checkpoint
model = Darknet(cfg).to(device)  # create model

ckpt = torch.load(weights, map_location=device)  # load checkpoint
ckpt['model'] = {k: v for k, v in ckpt['model'].items() if model.state_dict()[k].numel() == v.numel()}
model.load_state_dict(ckpt['model'], strict=False)

# Evaluation mode
model.eval()

Darknet(
  (module_list): ModuleList(
    (0): Sequential(
      (Conv2d): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (BatchNorm2d): BatchNorm2d(32, eps=0.0001, momentum=0.03, affine=True, track_running_stats=True)
      (activation): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (1): Sequential(
      (Conv2d): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (BatchNorm2d): BatchNorm2d(64, eps=0.0001, momentum=0.03, affine=True, track_running_stats=True)
      (activation): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (2): Sequential(
      (Conv2d): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (BatchNorm2d): BatchNorm2d(64, eps=0.0001, momentum=0.03, affine=True, track_running_stats=True)
      (activation): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (3): FeatureConcat_l()
    (4): Sequential(
      (Conv2d): Conv2d(32, 32, kernel_size=(3, 3), 

In [ ]:
img = torch.zeros((1, 3, imgsz, imgsz), device=device)  # init img
_ = model(img) if device.type != 'cpu' else None  # run once

### Load Data

In [12]:
imgsz = check_img_size(imgsz, s=64)
imgsz_test = check_img_size(imgsz_test, s=64)

with open(data) as f:
    data_dict = yaml.load(f, Loader=yaml.FullLoader)  # data dict

check_dataset(data_dict)  # check    
withrider_path = data_dict['with_rider']
withoutrider_path = data_dict['without_rider']
other_path = data_dict['other']
nc, names = (int(data_dict['nc']), data_dict['names'])  # number classes, names
assert len(names) == nc, '%g names found for nc=%g dataset in %s' % (len(names), nc, opt.data)  # check
print(names)

['animal', 'flat.driveable_surface', 'human.pedestrian.adult', 'human.pedestrian.child', 'human.pedestrian.construction_worker', 'human.pedestrian.personal_mobility', 'human.pedestrian.police_officer', 'human.pedestrian.stroller', 'human.pedestrian.wheelchair', 'movable_object.barrier', 'movable_object.debris', 'movable_object.pushable_pullable', 'movable_object.trafficcone', 'static_object.bicycle_rack', 'vehicle.bicycle', 'vehicle.bus.bendy', 'vehicle.bus.rigid', 'vehicle.car', 'vehicle.construction', 'vehicle.ego', 'vehicle.emergency.ambulance', 'vehicle.emergency.police', 'vehicle.motorcycle', 'vehicle.trailer', 'vehicle.truck']


In [13]:
withloader, withset = create_dataloader(withrider_path, imgsz, batch_size, gs)
mlc = np.concatenate(withset.labels, 0)[:, 0].max()  # max label class
nb = len(withloader)  # number of batches
assert mlc < nc, 'Label class %g exceeds nc=%g in %s. Possible class labels are 0-%g' % (mlc, nc, data, nc - 1)

Scanning labels /root/data/nuimages/labels/samples/CAM_BACK.cache3 (1092 found, 0 missing, 0 empty, 0 duplicate, for 1092 images): 1092it [00:00, 10647.15it/s]


In [67]:
withoutloader, withoutset = create_dataloader(withoutrider_path, imgsz_test, batch_size, gs, rect=True, pad=pad)  # testloader
nb1 = len(withoutloader)  # number of batches

Scanning images: 100%|██████████| 3141/3141 [00:40<00:00, 78.12it/s] 
Scanning labels /root/data/nuimages/labels/samples/CAM_BACK.cache3 (3141 found, 0 missing, 0 empty, 0 duplicate, for 3141 images): 3141it [00:00, 8271.82it/s]


In [68]:
otherloader, otherset = create_dataloader(other_path, imgsz_test, batch_size, gs, rect=True, pad=pad)  # testloader
nb2 = len(otherloader)  # number of batches

Scanning images: 100%|██████████| 9831/9831 [01:27<00:00, 112.06it/s]
Scanning labels /root/data/nuimages/labels/samples/CAM_BACK.cache3 (9822 found, 0 missing, 9 empty, 0 duplicate, for 9831 images): 9831it [00:01, 8652.63it/s]


### Predict Using Model

In [15]:
for batch_i, (img, targets, paths, shapes) in enumerate(tqdm(withloader)):
    print(targets)
    print(img.shape[0])
    break

  0%|          | 0/23 [00:00<?, ?it/s]

tensor([[0.00000e+00, 1.70000e+01, 7.64375e-01, 5.10937e-01, 1.87499e-02, 1.43752e-02],
        [0.00000e+00, 1.70000e+01, 9.41875e-01, 5.19063e-01, 7.62499e-02, 3.81251e-02],
        [0.00000e+00, 1.70000e+01, 9.82812e-01, 5.22500e-01, 3.43749e-02, 3.87500e-02],
        ...,
        [4.70000e+01, 1.70000e+01, 6.22500e-01, 5.16875e-01, 6.37500e-02, 4.50000e-02],
        [4.70000e+01, 1.70000e+01, 7.88125e-01, 5.58125e-01, 1.53750e-01, 1.17500e-01],
        [4.70000e+01, 2.20000e+01, 6.99063e-01, 5.00313e-01, 9.37492e-03, 1.81249e-02]])
48


In [17]:
stats = []
for batch_i, (img, targets, paths, shapes) in enumerate(tqdm(withloader)):
    img = img.to(device, non_blocking=True)
    img = img.float()  # uint8 to fp16/32
    img /= 255.0  # 0 - 255 to 0.0 - 1.0
    targets = targets.to(device)
    nb, _, height, width = img.shape  # batch size, channels, height, width
    with torch.no_grad():
        inf_out = model(img)[0]
        output = non_max_suppression(inf_out, conf_thres=conf_thres, iou_thres=iou_thres)
        accumulate_stats(stats, output, targets, nc, device, height, width)
stats = [np.concatenate(x, 0) for x in zip(*stats)]  # to numpy

100%|██████████| 23/23 [00:28<00:00,  1.24s/it]


In [40]:
p, r, ap, f1, ap_class = ap_per_class(*stats, plot=False)

In [89]:
unique_classes = np.unique(stats[3])
print('{:<36}\t{:<10}\t{:<10}\t{:<10}'.format('Class', 'Pricision@0.1', 'Recall@0.1', 'mAP for class'))
for i, cls in enumerate(unique_classes):
    print('{:<36}\t{:.10f}\t{:.10f}\t{:.10f}'.format(names[int(cls)], p[i][0], r[i][0], np.mean(ap[i])) )

print()

print('{:<36}\t{:<10}'.format('Class', 'AP@0.5:0.95'))
for i, cls in enumerate(unique_classes):
    print('{:<36}\t'.format(names[int(cls)]), end='')
    for j in ap[i]:
        print('{:5f}  '.format(j), end='')
    print()

print()
print('{:<10}\t{:<10}\t{:<10}'.format('Mean Pricision', 'Mean Recall', 'mAP'))
print('{:.10f}\t{:.10f}\t{:.10f}'.format(np.mean(p[:,0]), np.mean(r[:,0]), np.mean(ap)) )



Class                               	Pricision@0.1	Recall@0.1	mAP for class
animal                              	0.0000000000	0.0000000000	0.0162473876
human.pedestrian.adult              	0.4236908130	0.7513051732	0.3816287376
human.pedestrian.child              	0.1282785065	0.0666666667	0.0301323538
human.pedestrian.construction_worker	0.3343596420	0.6971153846	0.2828143693
human.pedestrian.personal_mobility  	0.1718048425	0.3571428571	0.1360895316
human.pedestrian.police_officer     	0.0000000000	0.0000000000	0.0108555106
human.pedestrian.stroller           	0.4405753237	0.7500000000	0.3715889199
human.pedestrian.wheelchair         	0.0000000000	0.0000000000	0.0000000000
movable_object.barrier              	0.3981260938	0.8001917774	0.4278869516
movable_object.debris               	0.0894851512	0.2121212121	0.0534920449
movable_object.pushable_pullable    	0.3092030145	0.7187500000	0.2375867268
movable_object.trafficcone          	0.4425842061	0.7648839556	0.4014044837
static_objec

In [ ]:
stats1 = []
for batch_i, (img, targets, paths, shapes) in enumerate(tqdm(withoutloader)):
    img = img.to(device, non_blocking=True)
    img = img.float()  # uint8 to fp16/32
    img /= 255.0  # 0 - 255 to 0.0 - 1.0
    targets = targets.to(device)
    nb, _, height, width = img.shape  # batch size, channels, height, width
    with torch.no_grad():
        inf_out = model(img)[0]
        output = non_max_suppression(inf_out, conf_thres=conf_thres, iou_thres=iou_thres)
        accumulate_stats(stats1, output, targets, nc, device, height, width)
stats1 = [np.concatenate(x, 0) for x in zip(*stats1)]  # to numpy

p1, r1, ap1, f11, ap_class1 = ap_per_class(*stats1, plot=False)


In [90]:
unique_classes = np.unique(stats1[3])
print('{:<36}\t{:<10}\t{:<10}\t{:<10}'.format('Class', 'Pricision@0.1', 'Recall@0.1', 'mAP for class'))
for i, cls in enumerate(unique_classes):
    print('{:<36}\t{:.10f}\t{:.10f}\t{:.10f}'.format(names[int(cls)], p1[i][0], r1[i][0], np.mean(ap1[i])) )

print()

print('{:<36}\t{:<10}'.format('Class', 'AP@0.5:0.95'))
for i, cls in enumerate(unique_classes):
    print('{:<36}\t'.format(names[int(cls)]), end='')
    for j in ap1[i]:
        print('{:5f}  '.format(j), end='')
    print()

print()
print('{:<10}\t{:<10}\t{:<10}'.format('Mean Pricision', 'Mean Recall', 'mAP'))
print('{:.10f}\t{:.10f}\t{:.10f}'.format(np.mean(p1[:,0]), np.mean(r1[:,0]), np.mean(ap1)) )

Class                               	Pricision@0.1	Recall@0.1	mAP for class
animal                              	0.0000000000	0.0000000000	0.0392087073
human.pedestrian.adult              	0.4719564612	0.7711167086	0.4158139107
human.pedestrian.child              	0.0768364838	0.0322580645	0.0079519900
human.pedestrian.construction_worker	0.3555503417	0.6106382979	0.2669061393
human.pedestrian.personal_mobility  	0.3471502881	0.7364532020	0.1864432467
human.pedestrian.police_officer     	0.0000000000	0.0000000000	0.0871143010
human.pedestrian.stroller           	0.4306182035	0.3125000000	0.2233681258
human.pedestrian.wheelchair         	0.0000000000	0.0000000000	0.0000000000
movable_object.barrier              	0.4450960625	0.8094599128	0.4559263467
movable_object.debris               	0.1571745852	0.2894303385	0.0705275874
movable_object.pushable_pullable    	0.3162797122	0.5207100592	0.2127787803
movable_object.trafficcone          	0.5201519425	0.8297872340	0.4781892598
static_objec

In [ ]:
stats2 = []
for batch_i, (img, targets, paths, shapes) in enumerate(tqdm(otherloader)):
    img = img.to(device, non_blocking=True)
    img = img.float()  # uint8 to fp16/32
    img /= 255.0  # 0 - 255 to 0.0 - 1.0
    targets = targets.to(device)
    nb, _, height, width = img.shape  # batch size, channels, height, width
    with torch.no_grad():
        inf_out = model(img)[0]
        output = non_max_suppression(inf_out, conf_thres=conf_thres, iou_thres=iou_thres)
        accumulate_stats(stats2, output, targets, nc, device, height, width)
stats2 = [np.concatenate(x, 0) for x in zip(*stats2)]  # to numpy

p2, r2, ap2, f12, ap_class2 = ap_per_class(*stats2, plot=False)

In [91]:
unique_classes = np.unique(stats2[3])
print('{:<36}\t{:<10}\t{:<10}\t{:<10}'.format('Class', 'Pricision@0.1', 'Recall@0.1', 'mAP for class'))
for i, cls in enumerate(unique_classes):
    print('{:<36}\t{:.10f}\t{:.10f}\t{:.10f}'.format(names[int(cls)], p2[i][0], r2[i][0], np.mean(ap2[i])) )

print()

print('{:<36}\t{:<10}'.format('Class', 'AP@0.5:0.95'))
for i, cls in enumerate(unique_classes):
    print('{:<36}\t'.format(names[int(cls)]), end='')
    for j in ap2[i]:
        print('{:5f}  '.format(j), end='')
    print()
    
print()
print('{:<10}\t{:<10}\t{:<10}'.format('Mean Pricision', 'Mean Recall', 'mAP'))
print('{:.10f}\t{:.10f}\t{:.10f}'.format(np.mean(p2[:,0]), np.mean(r2[:,0]), np.mean(ap2)) )

Class                               	Pricision@0.1	Recall@0.1	mAP for class
animal                              	0.0000000000	0.0000000000	0.0120988171
human.pedestrian.adult              	0.4933997453	0.7779603625	0.4173183529
human.pedestrian.child              	0.1603157216	0.1071428571	0.0282696817
human.pedestrian.construction_worker	0.3861860690	0.6338266385	0.2803426276
human.pedestrian.police_officer     	0.0000000000	0.0000000000	0.0299396035
human.pedestrian.stroller           	0.5346809532	0.2857142857	0.1658604040
movable_object.barrier              	0.4310119606	0.7902807074	0.4379173301
movable_object.debris               	0.1983619771	0.2938931298	0.0633600352
movable_object.pushable_pullable    	0.2502778847	0.4565826331	0.1539829404
movable_object.trafficcone          	0.5430291919	0.8364452680	0.4902008302
static_object.bicycle_rack          	0.3744589784	0.6993019664	0.2540254498
vehicle.bicycle                     	0.0018435699	0.5000000000	0.0008012618
vehicle.bus.

In [10]:
!nvidia-smi

Tue Dec 13 22:03:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA TITAN RTX    Off  | 00000000:01:00.0 Off |                  N/A |
| 41%   36C    P2    53W / 280W |  23790MiB / 24219MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA TITAN RTX    Off  | 00000000:02:00.0 Off |                  N/A |
| 41%   

In [ ]:
device = 'cuda:2'
p, r, ap, f1, ap_class, raw_stats, path =  mAP_dataset(model, withloader, device, names, nc, conf_thres, iou_thres)
